# Produce messages to a partitioned Apache Kafka topic

In this notebook you will learn how to produce messages to an Apache Kafka topic which has two partitions

![Produce messages to an Apache Kafka Topic partitioned](../img/produce-partition.png)

---

## Prerequisites

To start the tutorial you need to:

* Download the Aiven for Apache Kafka SSL certificates as mentioned in the [README instructions](../README.md#Download-the-required-SSL-certificates)
* Retrieve the Aiven for Apache Kafka hostname and port, from the [Aiven Console](https://console.aiven.io/) in the Aiven for Apache Kafka service overview
* Substitute the Apache Kafka hostname and port in the parameters below

In [ ]:
# Replace the following two placeholders with Aiven for Apache Kafka service hostname and port

hostname="<MYKAFKAHOST>"
port="<MYKAFKAPORT>"

---

### Install the required libraries

The following installs the [kafka-python library](https://kafka-python.readthedocs.io/en/master/) we'll use for the tutorial

In [ ]:
!pip install confluent-kafka

---
## Create a new topic with two partitions

In this section, we'll use the Kafka Admin APIs to create a new `pizzaPartitioned` topic with two partitions

In [ ]:
from confluent_kafka.admin import AdminClient, NewTopic

conf = {
    'bootstrap.servers': hostname+":"+port,
    'client.id': 'myclient',
    'security.protocol': 'SSL',
    'ssl.ca.location': '../sslcerts/ca.pem',
    'ssl.certificate.location': '../sslcerts/service.cert',
    'ssl.key.location': '../sslcerts/service.key'
    }

admin = AdminClient(conf)

topic=NewTopic(
    topic="pizzaPartitioned", 
    num_partitions=2, 
    replication_factor=1
    )

admin.create_topics(
    [topic]
    )

---

## Create an Apache Kafka producer

The next step is to configure the Kafka producer.

In [ ]:
import json
from confluent_kafka import SerializingProducer

def json_serializer(msg, s_obj):
    return json.dumps(msg).encode('ascii')

conf = {
    'bootstrap.servers': hostname+":"+port,
    'client.id': 'myclient',
    'security.protocol': 'SSL',
    'ssl.ca.location': '../sslcerts/ca.pem',
    'ssl.certificate.location': '../sslcerts/service.cert',
    'ssl.key.location': '../sslcerts/service.key', 
    'value.serializer': json_serializer,
    'key.serializer': json_serializer
    }

producer = SerializingProducer(conf)

---
## Push the messages to the two partitions in the `pizzaPartitioned` topic

In the below section we are pushing data to the two partitions of the `pizzaPartitioned` topic. 
This can be achieved in several ways:

* By default, Apache Kafka uses the message key to define the partitions, messages different keys will likely be assigned to different partitions. If the key is null a random partition (with stickiness) will be selected
* A producer can define a partition upfront by either declaring the partition or using a custom partitioner

We'll use the second method, declaring the custom partition

In [ ]:
producer.produce(
    "pizzaPartitioned",
    key={"id":1},
    value={"id":1, "name":"👨 Francesco", "pizza":"Margherita 🍕"},
    partition=0
)

producer.produce(
    "pizzaPartitioned",
    key={"id":2},
    value={"id":2, "name":"👩 Adele", "pizza":"Hawaii 🍕+🍍+🥓"},
    partition=1
)

producer.flush()

---

## Produce more messages



In [ ]:
producer.produce(
    "pizzaPartitioned",
    key={"id":3},
    value={"id":3, "name":"👦 Mark", "pizza":"Choccolate 🍕+🍫"},
    partition=0
)

producer.produce(
    "pizzaPartitioned",
    key={"id":2},
    value={"id":2, "name":"👩 Ugo", "pizza":"Hawaii 🍕+🍍+🥓"},
    partition=1
)

producer.flush()